In [ ]:
import re
import os
import json
import shutil
import scipy.stats
import scipy.signal
import scipy.ndimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import matplotlib.gridspec as gridspec

Put `coveval` folder into the path and import modules:

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from coveval import utils
from coveval import scoring
from coveval.connectors import covasim
from coveval.connectors import covidsim
from coveval.connectors import generic
from coveval.connectors import mc19
from coveval.core import smoothing
from coveval.core import normalising
from coveval.core import losses

Convenient utility to update the notebook when simultaneously working on the libraries:

In [ ]:
import importlib

In [ ]:
def reimport():
    importlib.reload(mc19)
    importlib.reload(covasim)
    importlib.reload(generic)
    importlib.reload(covidsim)
    importlib.reload(utils)
    importlib.reload(normalising)
    importlib.reload(losses)
    importlib.reload(smoothing)
    importlib.reload(scoring)

In [ ]:
reimport()

Notes:
- running all the cells in this notebook should allow you to reproduce the images in [./figs/](./figs/).
- we might talk about "Imperial" and "covidsim" interchangeably.
- we might use score and loss interchangeably: our score is a loss, so the lower the better.

# Retrieve reported data

Let's retrieve the latest data reported for the USA on https://covidtracking.com:

In [ ]:
reported_usa = utils.get_outbreak_data_usa()

Now let's look at number of reported daily fatalities (field `deathIncrease`) for each state of interest and experiment we different smoothing procedures.

In [ ]:
def show_state_reported_data(state, reported, smoothers, t_min='2020-03'):
    df = reported.loc[state].copy()
    for s_name, s in smoothers.items():
        s.smooth_df(df, 'deathIncrease', inplace=True)
        df.rename(columns={'deathIncrease_smoothed': 'deathIncrease_smoothed_' + s_name}, inplace=True)
        
    utils.show_data(df, cols=['deathIncrease_smoothed_missed','deathIncrease_smoothed_gaussian'],
                scatter=['deathIncrease'], t_min=t_min, y_label='daily fatalities', figsize=(12,5))

In [ ]:
example_smoothers = {'missed': smoothing.missed_cases(), 'gaussian': smoothing.gaussian(3)}

In [ ]:
# store the smoother for each geo in this dict
smoother = {}

## California

In [ ]:
show_state_reported_data('US-CA', reported_usa, example_smoothers)

In [ ]:
smoother['California'] = example_smoothers['gaussian']

## Illinois

In [ ]:
show_state_reported_data('US-IL', reported_usa, example_smoothers)

In [ ]:
smoother['Illinois'] = example_smoothers['gaussian']

## Massachusetts

In [ ]:
show_state_reported_data('US-MA', reported_usa, example_smoothers)

In [ ]:
smoother['Massachusetts'] = example_smoothers['gaussian']

## Michigan

In [ ]:
show_state_reported_data('US-MI', reported_usa, example_smoothers)

In [ ]:
smoother['Michigan'] = example_smoothers['gaussian']

## New_Jersey

In [ ]:
show_state_reported_data('US-NJ', reported_usa, example_smoothers)

In [ ]:
smoother['New_Jersey'] = example_smoothers['gaussian']

This is a good example of issues with the data reported that could be addressed by implementing a more robust filter:

In [ ]:
df = reported_usa.loc['US-NJ'].copy()
y = df['deathIncrease'].values
plt.figure(figsize=(8, 4))
plt.scatter(np.arange(len(y)),y, alpha=0.3, label='reported')
plt.plot(np.arange(len(y)),scipy.ndimage.gaussian_filter1d(y, sigma=3), c='r', label='gaussian filter')
plt.plot(np.arange(len(y)),scipy.ndimage.gaussian_filter1d(scipy.ndimage.median_filter(y, size=3), sigma=3), c='g', label='gaussian + median filter')
plt.legend()

For simplicity's sake we'll stick to the same Gaussian filter for now.

## New_York

In [ ]:
show_state_reported_data('US-NY', reported_usa, example_smoothers)

In [ ]:
smoother['New_York'] = example_smoothers['gaussian']

This is another case where a fancier filter could help:

In [ ]:
df = reported_usa.loc['US-NY'].copy()
y = df['deathIncrease'].values
plt.figure(figsize=(8, 4))
plt.scatter(np.arange(len(y)),y, alpha=0.3, label='reported')
plt.plot(np.arange(len(y)),scipy.ndimage.gaussian_filter1d(y, sigma=3), c='r', label='gaussian filter')
plt.plot(np.arange(len(y)),scipy.ndimage.gaussian_filter1d(scipy.ndimage.median_filter(y, size=3), sigma=3), c='g', label='gaussian + median filter')
plt.legend()

# Mappings

Geographies:

In [ ]:
usa_iso = {
    'California': 'US-CA',
    'Illinois': 'US-IL',
    'Massachusetts': 'US-MA',
    'Michigan': 'US-MI',
    'New_Jersey': 'US-NJ',
    'New_York': 'US-NY',
    'Pennsylvania': 'US-PA'
}

And corresponding interventions times, valid as of beginning of June 2020:

In [ ]:
interventions = {
    'California': ['2020-03-19','2020-03-31'],
    'Illinois': ['2020-03-17','2020-03-21'],
    'Massachusetts': ['2020-03-17','2020-03-24'],
    'Michigan': ['2020-03-16','2020-03-24','2020-06-01'],
    'New_Jersey': ['2020-03-18','2020-03-21','2020-05-13'],
    'New_York': ['2020-03-18','2020-03-22','2020-05-22'],
    'Pennsylvania': ['2020-03-16','2020-04-01']
}

# Imperial

For details on the Imperial data see this [README](../data/2020_04_30/covidsim/README.md) on how it was generated.

## Load calibration runs

Define some utilities functions to extract the contact reduction value CR from the folder name and build a unique id based on the R0 and CR values.

In [ ]:
def extract_cr(name, regex=r"_cr([\d]+).txt"):
    cr_regex = regex
    out = re.findall(cr_regex, name)
    if len(out) > 1:
        raise ValueError('Expected exactly 1 match, found more.')
    elif len(out) == 0:
        print(name)
        raise ValueError('Expected exactly 1 match, found none.')
    return out[0]

In [ ]:
def param_to_id(r0, cr):
    return 'R' + str(r0) + '_cr' + str(cr)

In [ ]:
imperial_folder = {'2020-04-30': '../data/2020_04_30/covidsim',
                   '2020-05-30': '../data/2020_05_30/covidsim'}

Load predictions for daily fatalities (`incDeath`) for each run in the grid, and organise them by unique parameters id (this can take a few minutes):

In [ ]:
imperial_raw = {}
imperial_runs = {}
imperial_params = {}
for cal in imperial_folder.keys():
    # load raw dict
    imperial_raw[cal] = covidsim.batch_load_predictions(imperial_folder[cal], cols=['incDeath'],
                                                        prefix='imperial_',
                                                        exclude=['_config_files', '_param_files'])

    # organise by ids and store params
    imperial_params[cal] = {}
    imperial_runs[cal] = {}
    for geo in imperial_raw[cal].keys():
        imperial_params[cal][geo] = {}
        imperial_runs[cal][geo] = {}
        for r0 in imperial_raw[cal][geo].keys():
            for param in imperial_raw[cal][geo][r0].keys():
                cr = extract_cr(param)
                imperial_params[cal][geo][param_to_id(r0, cr)] = {'r0': float(r0), 'cr': float(cr)}
                imperial_runs[cal][geo][param_to_id(r0, cr)] = imperial_raw[cal][geo][r0][param]
del imperial_raw

## Summarise them

Compute quantiles to summarise variations of predictions amongst repeats runs for a given (R0, CR) set:

In [ ]:
imperial_aggregated = {}
for cal in imperial_folder.keys():
    imperial_aggregated[cal] = {}
    for geo in imperial_runs[cal].keys():
        imperial_aggregated[cal][geo] = {}
        for _id in imperial_runs[cal][geo].keys():
            imperial_aggregated[cal][geo][_id] = utils.calc_quantiles_over_dfs(dfs=imperial_runs[cal][geo][_id],
                                                                               col='imperial_incDeath',
                                                                               q=[0.1, 0.5, 0.9])

## Add truth

Add the reported values `deathIncrease`:

In [ ]:
for cal in imperial_folder.keys():
    for geo in imperial_aggregated[cal].keys():
        reported_usa_geo = reported_usa.loc[(usa_iso[geo], 'deathIncrease')]
        for _id, df in imperial_aggregated[cal][geo].items():
            imperial_aggregated[cal][geo][_id] = utils.add_outbreak_data(df, reported_usa_geo)

## Smooth predictions

Let's use the Gaussian filter to smooth out `covidsim` predictions:

In [ ]:
imperial_pred_smoother = smoothing.gaussian(3)
cols = ['imperial_incDeath-0.1','imperial_incDeath-0.5','imperial_incDeath-0.9']
for cal in imperial_folder.keys():
    print(cal)
    for geo in imperial_aggregated[cal].keys():
        print('  ' + geo + '...')
        for _id, df in imperial_aggregated[cal][geo].items():
            for col in cols:
                imperial_pred_smoother.smooth_df(df, col, inplace=True)    

Let's have a quick look at the data (we pick an arbitrary state and set of parameters):

In [ ]:
cal = '2020-04-30'
geo = 'California'
utils.show_data(df= imperial_aggregated[cal][geo]['R2.2_cr90'],
                cols=['imperial_incDeath-0.5','imperial_incDeath-0.5_smoothed'], 
                scatter=['deathIncrease'],
                t_min='2020-03-03', t_max='2020-07-01',
                fill_between=[['imperial_incDeath-0.1_smoothed','imperial_incDeath-0.9_smoothed']],
                colors={'fill_between': ['r'],
                        'cols': {'imperial_incDeath-0.5_smoothed': 'r', 'imperial_incDeath-0.5': 'b'},
                        'scatter': {'deathIncrease': '#797979'}},
                title=cal,
                title_font={'fontsize': 16}, figsize=(12,5),
                show_times=[cal])

In [ ]:
cal = '2020-05-30'
geo = 'California'
utils.show_data(df=imperial_aggregated[cal][geo]['R2.2_cr90'],
                cols=['imperial_incDeath-0.5','imperial_incDeath-0.5_smoothed'], 
                scatter=['deathIncrease'],
                t_min='2020-03-03', t_max='2020-07-01',
                fill_between=[['imperial_incDeath-0.1_smoothed','imperial_incDeath-0.9_smoothed']],
                colors={'fill_between': ['r'],
                        'cols': {'imperial_incDeath-0.5_smoothed': 'r', 'imperial_incDeath-0.5': 'b'},
                        'scatter': {'deathIncrease': '#797979'}},
                title=cal,
                title_font={'fontsize': 16}, figsize=(12,5),
                show_times=[cal])

## Score predictions

Define scorers for each geo:

In [ ]:
scorers = {geo: scoring.scorer(smoother=smoother[geo],
                               normaliser=normalising.dynamic_scaling(weights=[1, 2, 3, 4, 5, 6, 7]),
                               loss=losses.poisson()) for geo in smoother.keys()}

For each set of parameters, compute score of the _median_ prediction up to the calibration date:

In [ ]:
imperial_results = {}
imperial_scores = {}
for cal in imperial_aggregated.keys():
    print(cal)
    imperial_results[cal] = {}
    imperial_scores[cal] = {}
    for geo in sorted(imperial_aggregated[cal].keys()):
        print('  ' + geo + '...')
        imperial_results[cal][geo] = {}
        imperial_scores[cal][geo] = {}
        for _id, df in imperial_aggregated[cal][geo].items():
            # get detail scoring results up to calibration data
            imperial_results[cal][geo][_id] = scorers[geo].score_df(df=df,
                                                                    col_truth='deathIncrease',
                                                                    col_pred='imperial_incDeath-0.5_smoothed',
                                                                    t_max=cal,
                                                                    inplace=True)
            # store score
            imperial_scores[cal][geo][_id] = round(imperial_results[cal][geo][_id]['score'], 6)

We can quickly check that only the relevant periods were taken into account for each calibration: 

In [ ]:
imperial_results['2020-04-30']['New_York']['R3.75_cr80']['idx'][-1]

In [ ]:
imperial_results['2020-05-30']['New_York']['R3.75_cr80']['idx'][-1]

## Identify parameters with best scores

Sort scores for a given geo to identify which set of `(R0,CR)` in the grid compared most favourably to the reported data. The below produces a list of tuples `('params_id', 'score')` sorted by ascending `score` values.

In [ ]:
imperial_scores_sorted = {}
for cal in imperial_aggregated.keys():
    imperial_scores_sorted[cal] = {}
    for geo in imperial_aggregated[cal].keys():
        imperial_scores_sorted[cal][geo] = sorted(imperial_scores[cal][geo].items(), key=lambda x: x[1])

Now we can find the average R0 and contact reduction parameters for the top n scores:

In [ ]:
def find_average_imperial_params(cal, n, precision=2):
    scores = imperial_scores_sorted[cal]
    out = {}
    for geo in scores.keys():
        _r0s = []
        _crs = []
        for e in scores[geo][:n]:
            p = imperial_params[cal][geo][e[0]]
            _r0s.append(p['r0'])
            _crs.append(p['cr'])
        out[geo] = {'r0': (round(np.mean(_r0s), precision), round(np.std(_r0s), precision)),
                    'cr': (round(np.mean(_crs), precision), round(np.std(_crs), precision))}
    return out

Let's see compute what the best, and average top-3, such values were for the models calibrated on data up to `2020-04-30`:

In [ ]:
find_average_imperial_params('2020-04-30', 1)

In [ ]:
find_average_imperial_params('2020-04-30', 3)

Let's see compute what the best, and average top-3, such values were for the models calibrated a month later, i.e. with data up to `2020-05-30` available:

In [ ]:
find_average_imperial_params('2020-05-30', 1)

In [ ]:
find_average_imperial_params('2020-05-30', 3)

## Highlight best scoring predictions in grid of results

In [ ]:
def build_imperial_grid(params, geo):
    r_values = set()
    cr_values = set()
    for v in params[geo].values():
        r_values.add(v['r0'])
        cr_values.add(v['cr'])
    
    out_headers = {'rows': ['R=' + str(r) for r in sorted(r_values)],
                   'cols': ['SD=' + str(int(cr)) + '%' for cr in sorted(cr_values)]}
    out = np.asarray([[param_to_id(r, int(cr)) for cr in sorted(cr_values)] for r in sorted(r_values)])        
    return out, out_headers

In [ ]:
def show_imperial_grid(cal, geo, t_max='2020-07-01'):
    _params, _headers = build_imperial_grid(imperial_params[cal], geo) 
    utils.batch_show_data(geo=geo,
                          params=_params,
                          data = imperial_aggregated[cal],
                          scores=imperial_scores[cal],
                          cols=['imperial_incDeath-0.5_smoothed','deathIncrease_smoothed'],
                          fill_between=[('imperial_incDeath-0.1_smoothed', 'imperial_incDeath-0.9_smoothed')],
                          scatter=['deathIncrease'],
                          colors={'fill_between': ['#FFB479'],
                                  'scatter': {'deathIncrease': '#85C5FF'},
                                  'cols': {'imperial_incDeath-0.5_smoothed': '#FFB479',
                                           'deathIncrease_smoothed': '#85C5FF'}},
                          highlight = 'min',
                          highlight_num=3,
                          figsize=1.1,
                          t_max=t_max,
                          show_times=interventions[geo] + [cal.replace('_','-')],
                          headers=_headers,
                          linewidths={cal: 3})

In order to sanity check the above let's visualise the predictions for each set of parameters and see whether the top-n pics according to our score look reasonable. Each row correspond to a given R0 value, and each column to the effectiveness of the social distancing (SD) measures in reducing the contact rates (i.e. if SD=25% this corresponds to contact rates of 0.75 compared to no social distancing).
- blue dots and lines: raw and smoothed truth respectively
- red line and shaded area: median prediction of `covidsim` model and 10-90% quantile range respectively
- thin vertical lines: interventions in the state
- thick vertical line: calibration date (i.e. the model had acess to data to the left of that line)
- the score of each model is in the title (remember our score is a loss, so the lower the better)

Running the commands below for each state should reproduce the figures in [./figs/covidsim_grid/](./figs/covidsim_grid).

In [ ]:
show_imperial_grid('2020-04-30', 'California')

In [ ]:
show_imperial_grid('2020-05-30', 'New_York')

# Covasim

For details on the Covasim data see this [README](../data/2020_04_30/covasim/README.md) on how it was generated.

## Load all calibration runs

Load predictions for daily fatalities (`new_deaths`) for each run:

In [ ]:
covasim_folder = {'2020-04-30': '../data/2020_04_30/covasim/',
                  '2020-05-30': '../data/2020_05_30/covasim/',}

In [ ]:
covasim_raw = {}
covasim_calibrations_params = {}
covasim_calibrations_sorted = {}
for cal in covasim_folder.keys():
    # load raw predictions
    covasim_raw[cal] = covasim.batch_load_predictions(covasim_folder[cal], filename='results.json', prefix='covasim_', cols=['new_deaths'])

    # load parameters and loss associated to each calibration
    covasim_calibrations_params[cal] = {}
    for geo in covasim_raw[cal].keys():
        covasim_calibrations_params[cal][geo] = {}
        for n_trials in covasim_raw[cal][geo].keys():
            covasim_calibrations_params[cal][geo][n_trials] = {}
            for n in covasim_raw[cal][geo][n_trials].keys():
                covasim_calibrations_params[cal][geo][n_trials][n] = {}
                with open(os.path.join(covasim_folder[cal], geo, n_trials, n, 'loss.json')) as f:
                    covasim_calibrations_params[cal][geo][n_trials][n].update(json.load(f))
                with open(os.path.join(covasim_folder[cal], geo, n_trials, n, 'calibrated_parameters.json')) as f:
                    covasim_calibrations_params[cal][geo][n_trials][n].update(json.load(f))

    # sort each calibration for a geo based on its mismatch
    covasim_calibrations_sorted[cal] = {}
    for geo in covasim_raw[cal].keys():
        covasim_calibrations_sorted[cal][geo] = {}
        for n_trials in covasim_raw[cal][geo].keys():
            covasim_calibrations_sorted[cal][geo][n_trials] = sorted(covasim_calibrations_params[cal][geo][n_trials].items(), key=lambda x: x[1]['mismatch'])

## Compute mean prediction within calibration replicas

We compute the mean of predictions made with the same parameters (one calibration = one set of parameters):

In [ ]:
covasim_calibrations_mean = {}
for cal in covasim_raw.keys():
    covasim_calibrations_mean[cal] = {}
    for geo in covasim_raw[cal].keys():
        covasim_calibrations_mean[cal][geo] = {}
        for n_trials in covasim_raw[cal][geo].keys():
            covasim_calibrations_mean[cal][geo][n_trials] = {}
            for n in covasim_raw[cal][geo][n_trials].keys():
                dfs = covasim_raw[cal][geo][n_trials][n].values()
                covasim_calibrations_mean[cal][geo][n_trials][n] = utils.calc_mean_over_dfs(dfs, 'covasim_new_deaths')

## Compute inter-calibration quantiles for mean prediction

If different calibrations have been performed, i.e. if different set of parameters were available, we compute inter-calibration quantiles, keeping only the `covasim_n_best` calibrations (as measured by the loss associated to each calibration):

In [ ]:
covasim_n_best = 5

In [ ]:
covasim_aggregated = {}
for cal in covasim_raw.keys():
    covasim_aggregated[cal] = {}
    for geo in covasim_raw[cal].keys():
        covasim_aggregated[cal][geo] = {}
        for n_trials in covasim_raw[cal][geo].keys():
            dfs = {}
            for e in covasim_calibrations_sorted[cal][geo][n_trials][:covasim_n_best]:
                n = e[0]
                dfs[n] = covasim_calibrations_mean[cal][geo][n_trials][n]
            covasim_aggregated[cal][geo][n_trials] = utils.calc_quantiles_over_dfs(dfs, col='covasim_new_deaths-mean', q=[0.2, 0.5, 0.8])

## Add truth

Add the reported values `deathIncrease`:

In [ ]:
for cal in covasim_aggregated.keys():
    for geo in covasim_aggregated[cal].keys():
        reported_usa_geo = reported_usa.loc[(usa_iso[geo], 'deathIncrease')]
        for _id, df in covasim_aggregated[cal][geo].items():
            covasim_aggregated[cal][geo][_id] = utils.add_outbreak_data(df, reported_usa_geo)

## Smooth predictions

Let's use the Gaussian filter (same as for `covidsim`) to smooth out covasim predictions:

In [ ]:
covasim_pred_smoother = smoothing.gaussian(3)
cols = ['covasim_new_deaths-mean-0.2','covasim_new_deaths-mean-0.5','covasim_new_deaths-mean-0.8']
for cal in covasim_aggregated.keys():
    for geo in covasim_aggregated[cal].keys():
        for _id, df in covasim_aggregated[cal][geo].items():
            for col in cols:
                covasim_pred_smoother.smooth_df(df, col, inplace=True)

Let's have a quick look at the data (again, we pick an arbitrary state and set of parameters):

In [ ]:
cal = '2020-04-30'
geo = 'New_York'
utils.show_data(covasim_aggregated[cal][geo]['500'],
                cols=['covasim_new_deaths-mean-0.5','covasim_new_deaths-mean-0.5_smoothed'], 
                scatter=['deathIncrease'],
                t_min='2020-03-03', t_max='2020-07-01',
                fill_between=[['covasim_new_deaths-mean-0.2_smoothed','covasim_new_deaths-mean-0.8_smoothed']],
                colors={'fill_between': ['#B4A9FF'],
                        'cols': {'covasim_new_deaths-mean-0.5': '#FFACF1',
                                 'covasim_new_deaths-mean-0.5_smoothed': '#B483FF'},
                        'scatter': {'deathIncrease': '#797979'}},
                title=cal,
                title_font={'fontsize': 16}, figsize=(12,5),
                show_times=[cal])

In [ ]:
cal = '2020-05-30'
geo = 'New_York'
utils.show_data(covasim_aggregated[cal][geo]['500'],
                cols=['covasim_new_deaths-mean-0.5','covasim_new_deaths-mean-0.5_smoothed'], 
                scatter=['deathIncrease'],
                t_min='2020-03-03', t_max='2020-07-01',
                fill_between=[['covasim_new_deaths-mean-0.2_smoothed','covasim_new_deaths-mean-0.8_smoothed']],
                colors={'fill_between': ['#B4A9FF'],
                        'cols': {'covasim_new_deaths-mean-0.5': '#FFACF1',
                                 'covasim_new_deaths-mean-0.5_smoothed': '#B483FF'},
                        'scatter': {'deathIncrease': '#797979'}},
                title=cal,
                title_font={'fontsize': 16}, figsize=(12,5),
                show_times=[cal])

# MC-19

For details on the MC-19 data see this [README](../data/2020_06_11/mc19/README.md). In particular note that, unlike for `covidsim` and `covasim`, in this case the calibration was performed internally by the model using the latest available data at the time (`2020-06-11`). We compare these predictions to those made by covadsim and covasim when calibrated with data up to `2020-05-30`: this is not entirely fair (as it gives MC-19 about 10 days more of historical data) but good enough for our purpose here.

## Load predictions

Load predictions for the daily number of fatalities (`dailyDeath`):

In [ ]:
mc19_folder = '../data/2020_06_11/mc19/'
mc19_predictions = mc19.batch_load_predictions(mc19_folder, filename='*.json', prefix='mc19_', exclude=['_queries'])

## Add truth

Add the reported values `deathIncrease`:

In [ ]:
for geo, df in mc19_predictions.items():
    reported_usa_geo = reported_usa.loc[(usa_iso[geo], 'deathIncrease')]
    mc19_predictions[geo] = utils.add_outbreak_data(df, reported_usa_geo)

Let's have a quick look at the data:

In [ ]:
geo = 'New_York'
utils.show_data(mc19_predictions[geo], 
                cols=['mc19_dailyDeath-percentile50'], 
                scatter=['deathIncrease'],
                fill_between=[('mc19_dailyDeath-percentile10','mc19_dailyDeath-percentile90'),
                              ('mc19_dailyDeath-percentile20','mc19_dailyDeath-percentile80')],
                t_min='2020-03-03', t_max='2020-07-01',
                colors={'cols': {'mc19_dailyDeath-percentile50': 'g'},
                        'scatter': {'deathIncrease': '#797979'},
                        'fill_between': ['#8AC277','#8AC277']},
                show_times=['2020-06-11'],
                title='2020-06-11',
                title_font={'fontsize': 16}, figsize=(12,5))

# Compare models to each other

Note: we use the terms `covidsim` and `Imperial` interchangeably.

## Add predictions to Imperial data

Let's add `covasim` and `MC-19` predictions to the dataframes containing those of `covidsim`:

In [ ]:
def add_to_imperial_predictions(dfs, to_add, cols=None):
    '''
    dfs : a dictionary {'param': DataFrame} corresponding to the calibration grid for Imperial
    to_add : a sequence of DataFrame to merge to each value of dfs.
    cols : [str] which columns of the values of dfs to keep.
    '''
    out = {}
    for _id, df in dfs.items():
        out[_id] = df
        if cols is not None:
            out[_id] = out[_id][cols]
        for df_to_add in to_add:
            out[_id] = out[_id].merge(df_to_add, left_index=True, right_index=True, how='outer')
    return out

In [ ]:
data_comparison = {}
for cal in ['2020-04-30','2020-05-30']:
    data_comparison[cal] = {}
    _imperial_cols = ['imperial_incDeath-0.1_smoothed', 'imperial_incDeath-0.5_smoothed',
                      'imperial_incDeath-0.9_smoothed', 'deathIncrease','deathIncrease_smoothed']
    _covasim_cols = ['covasim_new_deaths-mean-0.2_smoothed','covasim_new_deaths-mean-0.5_smoothed',
                     'covasim_new_deaths-mean-0.8_smoothed']
    _mc19_cols = ['mc19_dailyDeath-percentile10','mc19_dailyDeath-percentile20','mc19_dailyDeath-percentile50',
                  'mc19_dailyDeath-percentile80','mc19_dailyDeath-percentile90']
    for geo in imperial_aggregated[cal].keys():
        n_trials = '500'
        data_comparison[cal][geo] = add_to_imperial_predictions(dfs=imperial_aggregated[cal][geo],
                                                                to_add=[covasim_aggregated[cal][geo][n_trials][_covasim_cols],
                                                                        mc19_predictions[geo][_mc19_cols]],
                                                                cols=_imperial_cols)

## Compare with best Imperial prediction

Visualise other models predictions against a specified `covidsim` prediction (the one with the best score by default):

In [ ]:
def show_comparison_imperial(df, idm=True, mc19=False, hide_fb=False, **kwargs):
    """
    Plot different models predictions on the same graph.

    Parameters
    ----------
    df : DataFrame
        A DataFrame with a DatetimeIndex containing the data to plot. Should contain the columns
        ['imperial_incDeath-0.5_smoothed','deathIncrease_smoothed'] and if `hide_fb` is set to False also the
        columns ['imperial_incDeath-0.1_smoothed','imperial_incDeath-0.9_smoothed'].
    idm : boolean, optional
        If True show IDM data i.e. column 'covasim_new_deaths-mean-0.5_smoothed' and, if relevant, the columns
        ['covasim_new_deaths-mean-0.2_smoothed','covasim_new_deaths-mean-0.8_smoothed'].
    mc19 : boolean, optional
        If True show MC-19 data i.e. column 'mc19_dailyDeath-percentile50' and, if relevant, the columns
        ['mc19_dailyDeath-percentile10','mc19_dailyDeath-percentile90'].
    hide_fb : boolean, optional
        If True only show the median lines for each model.
    figsize : 2-tuple, optional
        The figure size.
    **kwargs : keyword arguments of `coveval.utils.show_data()`.

    Returns
    -------
    A matplotlib.axes object.
    """
    # specify columns to display
    scatter=['deathIncrease']
    cols=['imperial_incDeath-0.5_smoothed','deathIncrease_smoothed']
    fill_between=[('imperial_incDeath-0.1_smoothed','imperial_incDeath-0.9_smoothed')]
    if idm:
        cols.append('covasim_new_deaths-mean-0.5_smoothed')
        fill_between.append(('covasim_new_deaths-mean-0.2_smoothed','covasim_new_deaths-mean-0.8_smoothed'))
    if mc19:
        cols.append('mc19_dailyDeath-percentile50')
        fill_between.append(('mc19_dailyDeath-percentile10','mc19_dailyDeath-percentile90'))
    if hide_fb:
        fill_between=None
        
    return utils.show_data(df=df, cols=cols, fill_between=fill_between, scatter=scatter,
                           y_label='daily fatalities', **kwargs)

In [ ]:
def batch_show_comparison_best_imperial(cal, imperial_id=None, geos=None, n_cols=3, n_rows=None, leg_pos=-1,
                                        figsize=1.25, idm=True, mc19=False, t_min='2020-03', t_max=None,
                                        **kwargs):
    """
    Wrapper around `show_comparison_imperial()` that plots a grid comparing predictions by Imperial model and
    others for different geographies.
    
    Relies on the existence of the `data_comparison`, `imperial_scores_sorted` and `interventions`
    dictionaries.
    
    Parameters
    ----------
    cal : str
        Date of calibration that is the key to a {cal: {geo: {id: df}}} dictionary where id corresponds to the
        Imperial parameters id and df is a DataFrame with a DatetimeIndex containing the data to plot.
    imperial_id : str, optional
        If specified use this value for `id` in the dictionary mentioned above. If unspecified, 
    geos : [str], optional
        If specified only plot data for these geos.
    n_cols : int, optional
        Number of columns.
    n_rows : int, optional
        Number of rows. If unspecified, determined based on `n_cols` and `geos`.
    leg_pos : int, optional
        Index of plot under which to positional the legend, using the flattened grid representation.
    figsize : 2-tuple or float, optional
        If a tuple specifies directly the figure size. If a float scales the default figure size.
        If unspecified the figure size is determined automatically based on the number of rows and columns.
    **kwargs : keyword arguments of `show_comparison_imperial`.

    Returns
    -------
    A matplotlib.axes object.
    """
    # determine grid size
    if geos is None:
        geos = data_comparison[cal].keys() 
    if n_rows is None:
        n_rows = len(geos) // n_cols
        if len(geos) % n_cols != 0:
            n_rows += 1        
    if len(geos) > n_rows * n_cols:
        raise ValueError('too many geos to plot for the grid.')
        
    # determine figure size
    if figsize is None:
        figsize = 1
    if type(figsize) in [int, float]:
        figsize = (8 * n_cols * figsize, 4 * n_rows*figsize)    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
        
    # plot each geo
    for i, geo in enumerate(sorted(geos)):
        if imperial_id is None:
            _id = imperial_scores_sorted[cal][geo][0][0]
        # manual adjustment to scale in NJ to minimise impact due a mistake in the reported data
        y_max = 750 if geo == 'New_Jersey' else None
        lines = show_comparison_imperial(df=data_comparison[cal][geo][_id],
                                         idm=idm,
                                         mc19=mc19,
                                         t_min=t_min,
                                         t_max=t_max,
                                         colors={'fill_between': ['r','#B4A9FF','#8AC277'],
                                                 'cols': {'imperial_incDeath-0.5_smoothed':'r',
                                                          'covasim_new_deaths-mean-0.5_smoothed': '#942192',
                                                          'mc19_dailyDeath-percentile50': 'g',
                                                          'deathIncrease_smoothed': '#797979'},
                                                 'scatter': {'deathIncrease': '#797979'}},
                                         ax=axes.flatten()[i],
                                         title=geo,
                                         title_font={'fontsize': 16, 'fontweight': 'normal'},
                                         show_leg=False,
                                         linewidths={cal: 3},
                                         show_times=interventions[geo] + [cal],
                                         y_max=y_max,
                                         **kwargs)
    fig.suptitle(cal, fontsize=22)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    # add legend
    labels = {'imperial_incDeath-0.5_smoothed': 'covidsim (Imperial)',
              'covasim_new_deaths-mean-0.5_smoothed': 'covasim (IDM)',
              'mc19_dailyDeath-percentile50': 'MC-19',
              'deathIncrease_smoothed': 'reported (smoothed)',
              'deathIncrease': 'reported'}
    _h = []
    _l = []
    for label, line in sorted(lines.items(), key=lambda x: x[0]):
        _h.append(line)
        _l.append(labels[label])
    axes.flatten()[leg_pos].legend(handles=_h, labels=_l,loc='upper center', bbox_to_anchor=(0.5, -0.12),
                                   fontsize='x-large', ncol=len(lines.keys()))
    return

Running the commands below with `t_max='2020-07-13'` should reproduce the figures [./figs/2020-04-30_models.png](./figs/2020-04-30_models.png) and [./figs/2020-05-30_models.png](./figs/2020-05-30_models.png).

In [ ]:
batch_show_comparison_best_imperial('2020-04-30', t_max='2020-07-13', leg_pos=-2, figsize=1.25, mc19=False)

In [ ]:
batch_show_comparison_best_imperial('2020-05-30', t_max='2020-07-13', leg_pos=-2, figsize=1.25, mc19=True)

## Compare models performance w.r.t to truth

Assess how each model predictions compare to the truth, using 2 summary metrics that can be computed _up to_ and _from_ the calibration date:
- cumulative deaths delta: difference between the total number of fatalities predicted by the model and the (smoothed) total number of fatalities reported
- average difference to daily deaths: the mean relative difference between daily predicted and reported values (with some leeway of +/- 10 to prevent small values from having too big of an impact)

It is also interesting to look at the score for each model _up to_ the calibration, to see which one appeared more promising _then_.

Note: the score _from_ the calibration date can be useful too but is slightly less straightforward to interpret since the normalisation procedure makes uses of earlier data but any penalty incurred then isn't included. This would for instance be useful to calibrate specific intervention periods.

In [ ]:
def compute_metrics(df, t_min=None, t_max=None, idm=True, mc19=False, resolution=10, scorer=None):
    col_truth = 'deathIncrease'
    col_ref = 'deathIncrease_smoothed'
    cols = [col_truth,'imperial_incDeath-0.5_smoothed']
    if idm:
        cols.append('covasim_new_deaths-mean-0.5_smoothed')
    if mc19:
        cols.append('mc19_dailyDeath-percentile50') 
    out = {col: {} for col in cols}
    
    # compute diff dataframe
    idx = df.index[~df[col_ref].isnull()]
    df_diff = df.loc[idx, [col_ref] + cols].copy()
    for col in cols:
        # compute distance to col_ref given resolution
        delta = df_diff[col].values - df_diff[col_ref].values
        
        # if within resolutsion: perfect
        delta = np.where(np.abs(delta) <= resolution, 0, delta)
        
        # if prediction above, substract the resolution
        delta = np.where(delta > 0 , delta - resolution, delta)
        
        # if prediction below, add the resolution
        delta = np.where(delta < 0 , delta + resolution, delta)

        # compute corresponding % (+1 to make sure we don't divide by 0)
        df_diff[col + '_delta_pc'] = delta /( df_diff[col_ref] + 1) * 100
    out['df'] = df_diff
    
    # compute duration of specified period
    idx = df_diff.index
    if t_min is None:
        t_min = idx[0]
    if t_max is None:
        t_max = idx[-1]
    idx = df_diff.loc[t_min:t_max].index
    out['duration'] = str(pd.Timedelta(idx.values[-1] - idx.values[0]).days)

    # compute score
    if scorer is not None:
        # we 
        for col in cols:
            out[col]['score'] = scorer.score_df(df_diff, col_truth, col, t_min=t_min, t_max=t_max)['score']
    
    # compute delta cumulative value
    out['cumsum_ref'] = df_diff.loc[idx, col_ref].cumsum().values[-1]
    for col in cols:
        out[col]['cumsum_delta'] = df_diff.loc[idx, col].cumsum().values[-1] - out['cumsum_ref']
        out[col]['cumsum_delta_pc'] = round(out[col]['cumsum_delta'] / out['cumsum_ref'] * 100, 2)

    # compute relative % to truth
    for col in cols:
        out[col]['mean_abs_pc'] = round(df_diff.loc[idx, col + '_delta_pc'].abs().mean(), 2)

    return out

In [ ]:
def batch_compute_metrics(cal, t_min=None, t_max=None, idm=True, mc19=False, verbose=False):
    out = {}
    for geo in sorted(data_comparison[cal].keys()):
        # get the predictions corresponding to the best covidsim model
        df = data_comparison[cal][geo][imperial_scores_sorted[cal][geo][0][0]]
        out[geo] = compute_metrics(df, t_min=t_min, t_max=t_max, idm=idm, mc19=mc19, scorer=scorers[geo])
        
        #display results for geo
        if verbose:
            print(geo)
            print('  duration: ' + str(out[geo]['duration']) + ' days')
            for col in out[geo].keys():
                if col not in ['duration','df','cumsum_ref']:
                    print('  ' + col + ':')
                    print('     ', out[geo][col], '')
            print('')
    return out

In [ ]:
models_metrics = {}

Models calibrated with data up to `2020-04-30`:

In [ ]:
cal = '2020-04-30'
models_metrics[cal] = {}
models_metrics[cal]['before'] = batch_compute_metrics(cal=cal, t_max=cal, mc19=False)
models_metrics[cal]['after'] = batch_compute_metrics(cal=cal, t_min=cal, t_max='2020-06-15', mc19=False)

Models calibrated with data up to `2020-05-30` (we include `mc19` even though it actually benefited from slightly more recent data):

In [ ]:
cal = '2020-05-30'
models_metrics[cal] = {}
models_metrics[cal]['before'] = batch_compute_metrics(cal=cal, t_max=cal, mc19=True)
models_metrics[cal]['after'] = batch_compute_metrics(cal=cal, t_min=cal, t_max='2020-06-15', mc19=True)

As a sanity check, we can make sure that the loss up to calibration for `covidsim` is the same as the one computed earlier:

In [ ]:
cal = '2020-04-30'
geo = 'New_York'
print(models_metrics[cal]['before'][geo]['imperial_incDeath-0.5_smoothed']['score'])
print(imperial_scores_sorted[cal][geo][0][1])

## Summary

Show predictions and summary statistics for each model for each geography.

In [ ]:
def show_comparison_and_stats(cal, geo, idm=True, mc19=False, t_min='2020-03', t_max=None, t_boundary=None,
                              imperial_id=None, figsize=(19,5), axes=None, width_ratios=[1, 1.25], **kwargs):    
    col_names = {'imperial_incDeath-0.5_smoothed': 'Imperial',
                 'covasim_new_deaths-mean-0.5_smoothed': 'IDM',
                 'mc19_dailyDeath-percentile50': 'MC-19'}
    col_colours = {'imperial_incDeath-0.5_smoothed':'r',
                   'covasim_new_deaths-mean-0.5_smoothed': '#942192',
                   'mc19_dailyDeath-percentile50': 'g',
                   'deathIncrease_smoothed': '#797979'}    
    table_col_colours = {'imperial_incDeath-0.5_smoothed': '#FFD1D4',
                         'covasim_new_deaths-mean-0.5_smoothed': '#D0D0FF',
                         'mc19_dailyDeath-percentile50': '#D5F4CF',
                         'deathIncrease_smoothed': '#797979'}
    labels = {'imperial_incDeath-0.5_smoothed': 'Imperial (covidsim)',
              'covasim_new_deaths-mean-0.5_smoothed': 'IDM (covasim)',
              'mc19_dailyDeath-percentile50': 'Stanford (MC-19)',
              'deathIncrease_smoothed': 'reported smoothed',
              'deathIncrease': 'reported'}    
    if t_boundary is None:
        t_boundary = cal
    
    if axes is None:
        fig = plt.figure(figsize=figsize)
        gs = gridspec.GridSpec(2,2, width_ratios=width_ratios)
        axes = [plt.subplot(gs[:,0]), plt.subplot(gs[0,1]), plt.subplot(gs[1,1])]
    else:
        fig = None
    
    # graph
    #------
    if imperial_id is None:
        _id = imperial_scores_sorted[cal][geo][0][0]
    df = data_comparison[cal][geo][_id]
    lines = show_comparison_imperial(df=df,
                                 idm=idm,
                                 mc19=mc19,
                                 t_min=t_min,
                                 t_max=t_max,
                                 colors={'fill_between': ['r','#B4A9FF','#8AC277'],
                                         'cols': col_colours,
                                         'scatter': {'deathIncrease': '#797979'}},
                                 ax=axes[0],
                                 title=geo,
                                 title_font={'fontsize': 16, 'fontweight': 'normal'},
                                 show_leg=labels,
                                 linewidths={cal: 3},
                                 show_times=interventions[geo] + [cal],
                                 **kwargs)
    
    # statistics
    #-----------
    cols = ['deathIncrease','imperial_incDeath-0.5_smoothed']
    if idm:
        cols.append('covasim_new_deaths-mean-0.5_smoothed')
    if mc19:
        cols.append('mc19_dailyDeath-percentile50') 

    # table before
    metrics = compute_metrics(df, t_min=t_min, t_max=t_boundary, idm=idm, mc19=mc19, scorer=scorers[geo])
    row_1=['cumulative deaths delta (truth=' + str(int(metrics['cumsum_ref'])) + ')']
    row_2=['average difference to daily deaths']
    row_3=['scoring loss']
    colLabels=['UP TO calibration date']
    colColours=['#F2F2F2']
    for col in cols[1:]:
        row_1.append('{:+.2f}'.format(metrics[col]['cumsum_delta_pc']) + '% (' + '{:+d}'.format(int(metrics[col]['cumsum_delta'])) +')')
        row_2.append(str(metrics[col]['mean_abs_pc']) + '%')
        row_3.append(str(round(metrics[col]['score'],4)))
        colLabels.append(col_names[col])
        colColours.append(table_col_colours[col])
    axes[1].axis('off')
    table1 = axes[1].table(cellText=[row_1, row_2, row_3],
                           rowLabels=['','',''],
                           colLabels=colLabels,
                           colColours=colColours,
                           bbox=[0, 0, 1, 1],
                           colWidths=[0.8] + [0.35]*len(cols))
    table1.auto_set_font_size(False)
    table1.set_fontsize(12)

    # table after
    metrics = compute_metrics(df, t_min=t_boundary, t_max=t_max, idm=idm, mc19=mc19, scorer=None)
    row_1=['cumulative deaths delta (truth=' + str(int(metrics['cumsum_ref'])) + ')']
    row_2=['average difference to daily deaths']
    colLabels=['FROM calibration date']
    colColours=['#DBDBDB']
    for col in cols[1:]:
        row_1.append('{:+.2f}'.format(metrics[col]['cumsum_delta_pc']) + '% (' + '{:+d}'.format(int(metrics[col]['cumsum_delta'])) +')')
        row_2.append(str(metrics[col]['mean_abs_pc']) + '%')
        colLabels.append(col_names[col])
        colColours.append(table_col_colours[col])
    axes[2].axis('off')
    table2 = axes[2].table(cellText=[row_1, row_2],
                           rowLabels=['',''],
                           colLabels=colLabels,
                           colColours=colColours,
                           bbox=[0, 0, 1, 1],
                           colWidths=[0.8] + [0.35]*len(cols))
    table2.auto_set_font_size(False)
    table2.set_fontsize(12)

    if fig is not None:
        fig.tight_layout()

In [ ]:
def batch_show_comparison_and_stats(cal, width_ratios=[1, 1.25], figsize=(19,30), **kwargs):
    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(2*len(data_comparison[cal].keys()),2, width_ratios=width_ratios)
    for i, geo in enumerate(sorted(data_comparison[cal].keys())):
        axes = [plt.subplot(gs[2*i:2*(i+1),0]), plt.subplot(gs[2*i,1]), plt.subplot(gs[2*i+1,1])]
        # manual adjustment to scale in NJ to minimise impact due a mistake in the reported data
        y_max = 500 if geo == 'New_Jersey' else None
        show_comparison_and_stats(cal, geo, axes=axes, y_max=y_max, **kwargs)
    gs.tight_layout(fig, rect=[0, 0.03, 1, 0.97])
    fig.suptitle(cal, fontsize=22)
    return

Running the commands below with `t_max='2020-07-13'` should produce figures very similar to those in [./figs/2020-04-30_summary.png](./figs/2020-04-30_summary.png) and [./figs/2020-05-30_summary.png](./figs/2020-05-30_summary.png) (there maybe be small discrepancies due slightly different reported data values).

In [ ]:
batch_show_comparison_and_stats('2020-04-30', width_ratios=[1.1, 1], hide_fb=True, t_max='2020-07-13')

In [ ]:
batch_show_comparison_and_stats('2020-05-30', width_ratios=[1, 1.2], mc19=True, hide_fb=True, t_max='2020-07-13')